# Disk Fragmenter

In [1]:
EMPTY_BLOCK = -1

def get_disk_map(layout: str):
    return tuple([int(x) for x in list(layout.strip())])

def get_blocks(disk_map: tuple[int]):
    blocks = []
    file_id = 0
    for i in range(len(disk_map)):
        if i % 2 != 0:
            blocks.extend([EMPTY_BLOCK] * disk_map[i])
            continue

        blocks.extend([file_id] * disk_map[i])
        file_id += 1

    return blocks

def move_file_blocks(blocks: list[int]):
    clone = [ *blocks ]
    while True:
        first_free_index = clone.index(EMPTY_BLOCK)
        last_file_index = -1
        for i in range(len(clone)-1, 0, -1):
            if clone[i] != EMPTY_BLOCK:
                last_file_index = i
                break

        if first_free_index >= last_file_index:
            break

        clone[first_free_index] = clone[last_file_index]
        clone[last_file_index] = EMPTY_BLOCK

    return clone

def calc_checksum(moved_blocks: list[int]):
    out = 0
    for i in range(len(moved_blocks)):
        if moved_blocks[i] > EMPTY_BLOCK:
            out += i * moved_blocks[i]
    return out


with open("input.txt", "r") as f:
    layout = f.read()

disk_map = get_disk_map(layout)
blocks = get_blocks(disk_map)
moved_blocks = move_file_blocks(blocks)

calc_checksum(moved_blocks)

6288599492129

Correct: `6288599492129`

In [3]:
from colorama import Fore

def visualise(blocks: tuple[int]):
    print("".join([chr(block) if block > 9 else str(block) if block > -1 else "." for block in blocks]))


def show_fs(fs):
    out = ""
    for item in fs:
        blocks = [item["value"]] * item["len"]
        value = "".join([chr(block) if block > 9 else str(block) if block > -1 else "." for block in blocks])
        if item["stuck"]:
            value = Fore.BLUE + value + Fore.RESET;

        if item["moved"]:
            value = Fore.RED + value + Fore.RESET;
        out += value
    print(out)

def get_fs(blocks: list[int]):
    fs = []

    count = 0
    index = 0

    for i in range(len(blocks)):
        count += 1
        current = blocks[i]

        try:
            following = blocks[i+1]
        except IndexError:
            following = None

        defaults = {
            "len": count,
            "moved": False,
            "stuck": False
        }

        if current != following:
            if blocks[i] == EMPTY_BLOCK:
                fs.append({
                    "type": "free",
                    "value": EMPTY_BLOCK,
                    **defaults
                })

            if blocks[i] != EMPTY_BLOCK:
                fs.append({
                    "type": "file",
                    "value": blocks[i],
                    **defaults
                })
        
            count = 0
            index = i + 1
    return fs

def get_blocks_from_fs(fs):
    blocks = []
    for item in fs:
        blocks.extend([item["value"]] * item["len"])

    return blocks


def move_fs(fs, free_index, file_index):
    free = fs[free_index]
    file = fs[file_index]
    
    moved_file = {
        **file,
        "moved": True
    }
    released = {
        **file,
        "type": "free",
        "value": EMPTY_BLOCK,
        "moved": False,
        "stuck": False
    }
    
    fs[free_index] = moved_file
    fs[file_index] = released
    
    if free["len"] > file["len"]:
        fs.insert(free_index +1 , {
            **free,
            "len": free["len"] - file["len"]
        })

    return fs

def defragment(fs):
    reset = False
    while True:
        changed = False
        for rwd_idx in range(len(fs)-1, 0, -1):
            if changed:
                break

            if fs[rwd_idx]["type"] == "free":
                continue

            if fs[rwd_idx]["moved"] or fs[rwd_idx]["stuck"]:
                continue

            for fwd_idx in range(0, len(fs)):
                if fwd_idx >= rwd_idx:
                    break

                if fs[fwd_idx]["type"] == "file":
                    continue

                # free space
                if fs[fwd_idx]["len"] >= fs[rwd_idx]["len"]:
                    fs = move_fs(fs, fwd_idx, rwd_idx)
                    changed = True
                    break
            else:
                fs[rwd_idx]["stuck"] = True

        # show_fs(fs)
        if not changed:
            break

    return fs


# with open("test.txt", "r") as f:
#     layout = f.read()

with open("input.txt", "r") as f:
    layout = f.read()

disk_map = get_disk_map(layout)

blocks = get_blocks(disk_map)
# visualise(blocks)

updated_blocks = get_blocks_from_fs(defragment(get_fs(blocks)))
# print(updated_blocks)

calc_checksum(updated_blocks)

6321896265143

# Attempts

- `6321896265143` - **Correct!**
- `6410956071839` - That's not the right answer; your answer is too high.
- `6421391962883` - That's not the right answer
- `9185413613196` - That's not the right answer; your answer is too high.
- `10760448444221` - That's not the right answer; your answer is too high.